In [ ]:
import pandas as pd
file_path = "D:/100k.csv"
df = pd.read_csv(file_path)
df

C:\Users\Asus\AppData\Local\Temp\ipykernel_5824\1540798196.py:5: DtypeWarning: Columns (29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,Sentiment,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8,pos_9,...,pos_22,pos_23,pos_24,pos_25,pos_26,pos_27,pos_28,pos_29,pos_30,pos_31
0,negative,SCONJ,AUX,NOUN,ADP,NOUN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,negative,PRON,NOUN,ADV,SCONJ,VERB,ADP,ADJ,NOUN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,positive,INTJ,INTJ,PRON,AUX,ADV,PRON,NOUN,NOUN,CCONJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,negative,AUX,ADJ,ADJ,CCONJ,AUX,VERB,ADP,NOUN,VERB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,negative,PRON,VERB,AUX,ADJ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,positive,VERB,PRON,ADV,ADV,NOUN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99996,negative,PRON,AUX,VERB,PRON,NOUN,ADV,ADV,ADV,PRON,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99997,negative,AUX,ADJ,VERB,ADP,NOUN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99998,positive,NOUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score



pos_tags = ['AUX', 'CCONJ', 'INTJ', 'PRON', 'VERB', 'SCONJ', 'ADJ', 'NOUN', 'ADV', 'PROPN', 'ADP']
pos_columns = [f'pos_{i}' for i in range(1, 32)]

results = []

for size in range(1, 12):
    print(f"\n--- Subset Size: {size} ---")
    for combo in combinations(pos_tags, size):
        combination = set(combo)
        filtered_rows = []

        for _, row in df.iterrows():
            tags = row[pos_columns].tolist()
            filtered = [tag if tag in combination else np.nan for tag in tags]
            non_nan = [tag for tag in filtered if pd.notna(tag)]
            filtered_rows.append(non_nan)

        max_len = max(len(row) for row in filtered_rows)
        if max_len == 0:
            continue

        shifted_df = pd.DataFrame([
            row + [np.nan] * (max_len - len(row)) for row in filtered_rows
        ], columns=[f'pos_{i+1}' for i in range(max_len)])

        shifted_df['Sentiment'] = df['Sentiment'].values

        # Encode
        X_full = shifted_df.drop(columns=['Sentiment']).astype(str)
        y = shifted_df['Sentiment']
        label_encoders = {}
        for col in X_full.columns:
            le = LabelEncoder()
            X_full[col] = le.fit_transform(X_full[col])
            label_encoders[col] = le

        # Train/test split and model
        X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.2, random_state=42)
        model = CategoricalNB()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)

        print(f"Combo: {combo} -> Max POS: {max_len} -> Accuracy: {acc:.4f}")
        results.append({'combo': combo, 'accuracy': acc, 'max_pos': max_len})

# Top 10 overall
top_10 = sorted(results, key=lambda x: x['accuracy'], reverse=True)[:10]
print("\nTop 10 Combinations:")
for i, res in enumerate(top_10, 1):
    print(f"{i}. Tags: {res['combo']} -> Accuracy: {res['accuracy']:.4f} (Max POS: {res['max_pos']})")

# Best per size
best_per_size = {}
for res in results:
    size = len(res['combo'])
    if size not in best_per_size or res['accuracy'] > best_per_size[size]['accuracy']:
        best_per_size[size] = res

print("\nBest combination per subset size:")
for size in sorted(best_per_size):
    res = best_per_size[size]
    print(f"Size {size}: Tags {res['combo']} -> Accuracy: {res['accuracy']:.4f} (Max POS: {res['max_pos']})")


--- Subset Size: 1 ---
Combo: ('AUX',) -> Max POS: 8 -> Accuracy: 0.5299
Combo: ('CCONJ',) -> Max POS: 4 -> Accuracy: 0.5085
Combo: ('INTJ',) -> Max POS: 7 -> Accuracy: 0.5152
Combo: ('PRON',) -> Max POS: 12 -> Accuracy: 0.5106
Combo: ('VERB',) -> Max POS: 11 -> Accuracy: 0.5170
Combo: ('SCONJ',) -> Max POS: 4 -> Accuracy: 0.5069
Combo: ('ADJ',) -> Max POS: 7 -> Accuracy: 0.5042
Combo: ('NOUN',) -> Max POS: 14 -> Accuracy: 0.5162
Combo: ('ADV',) -> Max POS: 9 -> Accuracy: 0.5196
Combo: ('PROPN',) -> Max POS: 16 -> Accuracy: 0.5210
Combo: ('ADP',) -> Max POS: 8 -> Accuracy: 0.5073

--- Subset Size: 2 ---
Combo: ('AUX', 'CCONJ') -> Max POS: 10 -> Accuracy: 0.5296
Combo: ('AUX', 'INTJ') -> Max POS: 8 -> Accuracy: 0.5330
Combo: ('AUX', 'PRON') -> Max POS: 16 -> Accuracy: 0.5246
Combo: ('AUX', 'VERB') -> Max POS: 12 -> Accuracy: 0.5310
Combo: ('AUX', 'SCONJ') -> Max POS: 8 -> Accuracy: 0.5282
Combo: ('AUX', 'ADJ') -> Max POS: 10 -> Accuracy: 0.5296
Combo: ('AUX', 'NOUN') -> Max POS: 14 -> 